In [1]:
import tensorflow as tf


# import neptune
# from neptune.integrations.tensorflow_keras import NeptuneCallback

import os

import numpy as np
import pandas as pd
from window_generator_categorical import WindowGenerator

from run_automation_utils import Params, init_model

from typing import Dict

In [2]:
def prep_window_generator(station_id):
    df = pd.read_feather(f"../../data/pollution/processed/categorised/pm10/{station_id}.feather")
    df.set_index("timestamp", inplace=True)

    timestamp_s = df.index.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    year = (365.2425) * day

    df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
    df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
    df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
    df.reset_index(inplace=True)

    # Convert to radians.
    wd_rad = df.pop("winddirection_10m") * np.pi / 180
    df["winddirection_10m_sin"] = np.sin(wd_rad)
    df["winddirection_10m_cos"] = np.cos(wd_rad)

    df["kategorie"] = df["kategorie"].astype(np.int32)

    return WindowGenerator(input_width=24 * 7, label_width=24, shift=24, df=df.copy(), label_columns=["kategorie"])

In [3]:
window_generator_538 = prep_window_generator(538)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-17 15:19:22.414774: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-17 15:19:22.415182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## TODO: is that needed? train one with and one without (cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]))))


In [7]:
CONV_WIDTH = 3
OUT_STEPS = 24  # 14 days prediction
pred_categories = 6  # TODO: ??? num_features is 11 but must be one for our prediction

cnn_lstm_model = tf.keras.models.Sequential()
# Shape [batch, time, features] => [batch, CONV_WIDTH, features]
#cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :])) --> TODO: is that needed? train one with and one without
# Shape => [batch, 1, conv_units]
cnn_lstm_model.add(tf.keras.layers.Conv1D(256, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))

#cnn_lstm_model.add(tf.keras.layers.Flatten())
# LSTM
cnn_lstm_model.add(tf.keras.layers.LSTM(32, return_sequences=True))
cnn_lstm_model.add(tf.keras.layers.LSTM(32, return_sequences=False))

cnn_lstm_model.add(tf.keras.layers.Dense(512))
cnn_lstm_model.add(tf.keras.layers.Dropout(0.4))
# Shape => [batch, out_steps, features] -> for each prediction step one neuron
# Anpassung für die Klassifizierung
cnn_lstm_model.add(tf.keras.layers.Dense(OUT_STEPS * pred_categories, activation="softmax"))

# Shape => [batch, out_steps, pred_categories]
cnn_lstm_model.add(tf.keras.layers.Reshape([OUT_STEPS, pred_categories]))

In [8]:
cnn_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=["accuracy"]
) 

In [9]:
cnn_lstm_model.fit(window_generator_538.train, epochs=10)

(None, 24, 1)
Epoch 1/10


2023-06-17 14:58:07.431239: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 14:58:07.966364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 14:58:08.096666: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 14:58:08.504821: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 14:58:08.718926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


682/682 [==============================] - 35s 46ms/step - loss: 0.7445 - accuracy: 0.6644
Epoch 2/10
682/682 [==============================] - 32s 47ms/step - loss: 0.6946 - accuracy: 0.6855
Epoch 3/10
682/682 [==============================] - 32s 47ms/step - loss: 0.6780 - accuracy: 0.6917
Epoch 4/10
682/682 [==============================] - 31s 45ms/step - loss: 0.6610 - accuracy: 0.7011
Epoch 5/10
682/682 [==============================] - 32s 47ms/step - loss: 0.6484 - accuracy: 0.7081
Epoch 6/10
682/682 [==============================] - 31s 46ms/step - loss: 0.6332 - accuracy: 0.7152
Epoch 7/10
682/682 [==============================] - 31s 46ms/step - loss: 0.6240 - accuracy: 0.7202
Epoch 8/10
682/682 [==============================] - 35s 51ms/step - loss: 0.6104 - accuracy: 0.7267
Epoch 9/10
682/682 [==============================] - 32s 46ms/step - loss: 0.5870 - accuracy: 0.7392
Epoch 10/10
682/682 [==============================] - 32s 47ms/step - loss: 0.5651 - accurac

In [7]:
cnn_lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 166, 256)          9472      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 83, 256)          0         
 1D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 83, 256)          1024      
 hNormalization)                                                 
                                                                 
 conv1d_4 (Conv1D)           (None, 81, 512)           393728    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 40, 512)          0         
 1D)                                                             
                                                      

In [14]:
cnn_lstm_model.save("models/cnn_lstm_7days_history_1days_pred_10_epochs_2_lstm-layer-classification")

INFO:tensorflow:Assets written to: models/cnn_lstm_7days_history_1days_pred_10_epochs_2_lstm-layer-classification/assets


INFO:tensorflow:Assets written to: models/cnn_lstm_7days_history_1days_pred_10_epochs_2_lstm-layer-classification/assets


In [4]:
cnn_lstm_model = tf.keras.models.load_model("models/cnn_lstm_7days_history_1days_pred_10_epochs_2_lstm-layer-classification")

In [8]:
window_generator_538.train.element_spec

(None, 24, 1)


(TensorSpec(shape=(None, 168, 12), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 24, 1), dtype=tf.float64, name=None))

In [5]:
pred = cnn_lstm_model.predict(window_generator_538.train.take(1))

(None, 24, 1)


2023-06-17 15:19:53.514321: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-17 15:19:53.869872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 15:19:54.075154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-17 15:19:54.257790: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step


In [6]:
[p.argmax() for  p in pred[1]]

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
window_generator_538.plot(cnn_lstm_model, plot_col="kategorie", offset=32)

(3, 24, 1)
